# Analise de Embeddings Huggingface
Análise de modelos de embedding do Huggingfaces para msmarco

In [ ]:
pip install numpy faiss-cpu pandas sentence-transformers langchain langchain_community scikit-learn tqdm

In [2]:
import numpy as np
import faiss
import pandas as pd
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from sklearn.metrics import ndcg_score
import os
import csv
from typing import List, Tuple, Union, Dict, Any, Optional
from tqdm import tqdm


# Modelos de embedding
models = {
    "Portuguese-BGE-M3": "nonola/portuguese-bge-m3",
    "BAAI-BGE-M3": "BAAI/bge-m3",
    "MiniLM": "sentence-transformers/all-MiniLM-L6-v2"
}



In [3]:
def carregar_modelo(nome_modelo) -> HuggingFaceEmbeddings:
    if nome_modelo in models:
        return HuggingFaceEmbeddings(model_name=models[nome_modelo])
    else:
        raise ValueError(f"Modelo não suportado: {nome_modelo}")

In [4]:
def testar_embedding(documentos, modelo):
  embeddings = modelo.embed_documents([doc.page_content for doc in documentos[:10]]) # Certifique-se que o modelo está gerando os embeddings corretamente
  print(documentos[:10])
  if not embeddings:
    raise ValueError("A lista de embeddings está vazia.")
  else:
    print(embeddings)


In [5]:
def gravar_vetor_em_arquivo(vetor: List[Union[str, Tuple[str, Union[str, List[str]]]]], nome_arquivo: str) -> None:
    """
    Grava um vetor em um arquivo de texto, formatando os itens de acordo com seus tipos.

    A função suporta três tipos de itens no vetor:
    1.  Strings simples: gravadas diretamente no arquivo.
    2.  Tuplas (pergunta, resposta): gravadas como "Pergunta: pergunta" e "Resposta: resposta".
    3.  Tuplas (pergunta, lista de respostas): gravadas como "Pergunta: pergunta" e várias linhas "Resposta: resposta" para cada item na lista.

    Args:
        vetor: Uma lista contendo strings, tuplas de string ou tuplas de string e lista de string.
        nome_arquivo: O nome do arquivo onde o vetor será gravado.
    """
    with open(nome_arquivo, "w") as arquivo:
        for item in vetor:
            if isinstance(item, tuple) and isinstance(item[1], list):  # Se for uma tupla onde o segundo item é lista
                arquivo.write(f"Pergunta: {item[0]}\n")
                for resposta in item[1]:
                    arquivo.write(f"Resposta: {resposta}\n")  # Adiciona quebra de linha corretamente
                arquivo.write("\n")  # Adiciona espaçamento entre blocos
            elif isinstance(item, tuple):  # Se for uma tupla simples (pergunta, resposta única)
                arquivo.write(f"Pergunta: {item[0]}\n")
                arquivo.write(f"Resposta: {item[1]}\n\n")  # Dupla quebra para espaçamento
            else:  # Caso seja um item simples
                arquivo.write(f"{item}\n")

In [6]:
def contar_tokens(texto: str, modelo: HuggingFaceEmbeddings) -> int:
    """Conta a quantidade de tokens no texto usando o tokenizer do modelo."""
    tokenizer = modelo.client.tokenizer  # Acessa o tokenizer do modelo
    return len(tokenizer.encode(texto))

In [7]:
# Função para criar documentos com metadados
def criar_documento(texto, doc_id) -> Document :
    return Document(page_content=texto, metadata={"doc_id": doc_id, "original": texto})

# Função para criar FAISS com metadados
def indexar_faiss(documentos: List[Document], modelo: HuggingFaceEmbeddings , caminho_index="faiss_index") -> FAISS:
  """
    Cria e salva um índice FAISS a partir de documentos.

    Args:
        documentos: Lista de objetos Documento a serem indexados.
        modelo: Modelo de embeddings para vetorizar os documentos.
        caminho_index: Caminho para salvar o índice FAISS.

    Returns:
        O objeto FAISS criado.
  """
  print("Criando índice FAISS...")
  # Contagem total de tokens
  total_tokens = sum(contar_tokens(doc.page_content, modelo) for doc in documentos)
  print(f"Total de tokens: {total_tokens}")
  vectorstore = FAISS.from_documents(documentos, modelo)
  vectorstore.save_local(caminho_index)
  return vectorstore

# Função para carregar índice FAISS salvo
def carregar_faiss(caminho_index, modelo) -> FAISS:
    if os.path.exists(caminho_index):
        return FAISS.load_local(caminho_index, modelo)
    return None

def buscar_faiss(vectorstore: FAISS, query: str, top_k: int = 5) -> List[Tuple[str, Dict[str, Any]]]:
    """
    Realiza uma busca de similaridade em um índice FAISS.

    A função recebe um índice FAISS, uma query de busca e um número opcional de resultados a retornar.
    Retorna uma lista de tuplas, onde cada tupla contém o conteúdo do documento encontrado e seus metadados.

    Args:
        vectorstore: O índice FAISS onde a busca será realizada.
        query: A query de busca.
        top_k: O número de resultados a retornar (padrão: 5).

    Returns:
        Uma lista de tuplas, onde cada tupla contém:
            - O conteúdo do documento (str).
            - Um dicionário com os metadados do documento (Dict[str, Any]).
    """
    resultados: List[Document] = vectorstore.similarity_search(query, k=top_k)
    return [(res.page_content, res.metadata) for res in resultados]



In [8]:
def obter_conteudo_original_por_doc_id(documentos: List[Document], doc_id_desejado: int) -> Optional[str]:
    """
    Obtém o conteúdo original de um documento pelo doc_id.

    Args:
        documentos: Uma lista de objetos Document.
        doc_id_desejado: O doc_id do documento desejado.

    Returns:
        O conteúdo original do documento ou None se o doc_id não for encontrado.
    """
    for doc in documentos:
        if str(doc.metadata.get("doc_id")) == str(doc_id_desejado):
            return doc.metadata.get("original")
    raise ValueError(f"Nenhum documento encontrado com doc_id: {doc_id_desejado}")
    return None

In [9]:
# Carregar subconjunto do MS MARCO com ranqueamento BM25
def carregar_msmarco(caminho_collection, caminho_queries, caminho_qrels, caminho_bm25, num_queries=1000, num_posicoes_ranking=5) -> Tuple[List[Document], pd.DataFrame, pd.DataFrame]:
    # Carregar documentos e queries
    print("Carregando dados...")
    df_docs = pd.read_csv(caminho_collection, sep='\t', quoting=csv.QUOTE_NONE,header=None, names=['doc_id', 'document'])
    df_queries = pd.read_csv(caminho_queries, sep='\t', header=None, names=['query_id', 'query'])
    df_qrels = pd.read_csv(caminho_qrels, sep='\t', header=None, names=['query_id', '0', 'doc_id', 'relevance'])
    df_bm25 = pd.read_csv(caminho_bm25, sep='\t', header=None, names=['query_id', 'doc_id', 'rank'])

    # Filtrar apenas queries relevantes
    print("Filtrando dados...")
    df_qrels = df_qrels[df_qrels['relevance'] > 0]  # Apenas docs relevantes
    df_queriesl_filtered = df_queries[df_queries['query_id'].isin(df_qrels['query_id'])]
    # Amostrar queries
    sampled_queries = df_queriesl_filtered.sample(n=min(num_queries, len(df_queriesl_filtered)), random_state=42)

    # Criar conjunto de documentos a partir de BM25
    print("Criando conjunto de documentos a partir de BM25...")
    df_bm25_filtered = df_bm25[df_bm25['query_id'].isin(sampled_queries['query_id'])]
    df_bm25_filtered = df_bm25_filtered[df_bm25_filtered['rank'] <= num_posicoes_ranking]  # Pegamos os 5 primeiros do ranking
    df_docs_filtered = df_docs[df_docs['doc_id'].isin(df_bm25_filtered['doc_id'])]

    # Identificar IDs faltantes, já que o BM25 não é um gabarito, mas a execução de um modelo
    gabarito = df_qrels[df_qrels['query_id'].isin(sampled_queries['query_id'])]
    doc_ids_gabarito = set(gabarito['doc_id'])
    doc_ids_filtered = set(df_docs_filtered['doc_id'])
    doc_ids_faltantes = doc_ids_gabarito - doc_ids_filtered

    df_docs['doc_id'] = df_docs['doc_id'].astype(int)
    # Adicionar IDs faltantes a df_docs_filtered
    if doc_ids_faltantes:
        print(f"Adicionando {len(doc_ids_faltantes)} doc_ids faltantes a df_merged.")
        df_faltantes = df_docs[df_docs['doc_id'].isin(doc_ids_faltantes)]
        df_docs_filtered = pd.concat([df_docs_filtered, df_faltantes], ignore_index=True)

    # Criar documentos para FAISS
    print("Criando documentos para FAISS...")

    documentos = [criar_documento(row['document'], row['doc_id']) for _, row in df_docs_filtered.iterrows()]
    print(f"Total de documentos: {len(documentos)}")
    print(f"Total de queries: {len(sampled_queries)}")

    return documentos, sampled_queries, gabarito

In [10]:
def avaliar_modelo(nome_modelo: str, docs: List[Document], vectorstore: FAISS, queries: pd.DataFrame,
                   gabarito: pd.DataFrame, top_k=5) -> Dict[str, float]:
    """
    Avalia um modelo de busca utilizando métricas MRR, Recall@5 e NDCG@10.
    """
    print(f"Avaliando modelo... {nome_modelo}")

    estatisticas = {
        "acertos_completos": [],
        "acertos_parciais_pos_2": [],
        "acertos_parciais_pos_3": [],
        "acertos_parciais_outros": [],
        "erros": []
    }

    mrr, recall, ndcg = 0, 0, 0
    num_queries = len(queries)

    for _, row in tqdm(queries.iterrows(), total=len(queries), desc="processando queries"):
        query_id, query = row['query_id'], row['query']
        resultados = buscar_faiss(vectorstore, query, top_k)
        retrieved_ids = [res[1]['doc_id'] for res in resultados]
        ids_gabarito = gabarito[gabarito['query_id'] == query_id]['doc_id'].tolist()

        processar_resultado(query, ids_gabarito, retrieved_ids, resultados, docs, estatisticas)

        # Calculando métricas
        if any(id_correto in retrieved_ids for id_correto in ids_gabarito):
          first_correct_rank = min(retrieved_ids.index(id_correto) + 1 for id_correto in ids_gabarito if id_correto in retrieved_ids)
          mrr += 1 / first_correct_rank
        else:
          mrr += 0
        recall += len(set(retrieved_ids) & set(ids_gabarito)) / len(ids_gabarito)
        ndcg += ndcg_score([[1 if doc in ids_gabarito else 0 for doc in retrieved_ids]], [[1] * len(retrieved_ids)])

    metricas = {
        "MRR@10": mrr / num_queries,
        "Recall@5": recall / num_queries,
        "NDCG@10": ndcg / num_queries
    }
    salvar_resultados(nome_modelo, estatisticas, metricas)

    return metricas


In [11]:
def processar_resultado(query: str, ids_gabarito: List[int], retrieved_ids: List[int], resultados:  List[Tuple[str, Dict[str, Any]]], docs: List[Document], estatisticas: Dict[str, float]):
    """ Processa os resultados e classifica como acerto completo, parcial ou erro. """
    if retrieved_ids[0] in ids_gabarito:
        estatisticas["acertos_completos"].append((query, resultados[0][1]['original']))
    elif set(ids_gabarito) & set(retrieved_ids): #existe interseção entre os dois vetores
        set_gabarito = set(ids_gabarito)  # Para busca eficiente
        posicao =  next((i for i, x in enumerate(retrieved_ids) if x in set_gabarito), -1)
        lista_retorno = [res[1]['original'] for res in resultados[:posicao + 1]]
        lista_retorno[posicao] = '-->' + lista_retorno[posicao]
        if posicao == 1:
            estatisticas["acertos_parciais_pos_2"].append((query, lista_retorno))
        elif posicao == 2:
            estatisticas["acertos_parciais_pos_3"].append((query, lista_retorno))
        else:
            estatisticas["acertos_parciais_outros"].append((query, lista_retorno))
    else:
        lista_retorno_erro = [res[1]['original'] for res in resultados]
        for id_resposta_correta in ids_gabarito:
          lista_retorno_erro.append('-->' + obter_conteudo_original_por_doc_id(docs, id_resposta_correta))
        estatisticas["erros"].append((query, lista_retorno_erro))


In [12]:
def salvar_resultados(nome_modelo: str, estatisticas: Dict[str, List], metricas: Dict[str,float]):
    """ Salva os resultados da avaliação em arquivos. """
    resultado_texto = (f"Acertos Completos: {len(estatisticas['acertos_completos'])}\n"
                       f"Acertos Parciais Posição 2: {len(estatisticas['acertos_parciais_pos_2'])}\n"
                       f"Acertos Parciais Posição 3: {len(estatisticas['acertos_parciais_pos_3'])}\n"
                       f"Acertos Parciais Outros: {len(estatisticas['acertos_parciais_outros'])}\n"
                       f"Erros: {len(estatisticas['erros'])}\n"
                       f"Métricas: {metricas}")

    with open(f"resultado_{nome_modelo}.txt", "w") as arquivo:
        arquivo.write(resultado_texto)

    for categoria, vetor in estatisticas.items():
        gravar_vetor_em_arquivo(vetor, f"{categoria}.txt")


In [ ]:
# Caminhos dos arquivos (ajustar conforme necessário)
caminho_collection = "/content/drive/MyDrive/dataset/nlp/semantica/msmarco/portuguese_collection.tsv"
caminho_queries = "/content/drive/MyDrive/dataset/nlp/semantica/msmarco/portuguese_queries.dev.tsv"
caminho_qrels = "/content/drive/MyDrive/dataset/nlp/semantica/msmarco/qrels.dev.tsv"
caminho_bm25 = "/content/drive/MyDrive/dataset/nlp/semantica/msmarco/run.bm25_portuguese-msmarco.txt"
caminho_index = "faiss_index"
#nome_modelo = "MiniLM"
#nome_modelo = "Portuguese-BGE-M3";
nome_modelo = "BAAI-BGE-M3"
numero_posicoes_ranking_para_analise = 5

print(f"============= Análise modelo {nome_modelo} ==========================")

modelo = carregar_modelo(nome_modelo)

# Carregar dados
documentos,queries, relevancias = carregar_msmarco(caminho_collection, caminho_queries,
                                                    caminho_qrels, caminho_bm25,
                                                    num_queries=10000,
                                                    num_posicoes_ranking=numero_posicoes_ranking_para_analise)

# testar_embedding(documentos, modelo)

# Verificar se FAISS já foi indexado
vectorstore = carregar_faiss(caminho_index, modelo)
if vectorstore is None:
    vectorstore = indexar_faiss(documentos, modelo, caminho_index + "_" + nome_modelo)

# Avaliando modelo
metricas = avaliar_modelo(nome_modelo,
                          documentos,
                          vectorstore,
                          queries,
                          relevancias,
                          top_k=numero_posicoes_ranking_para_analise)

print(f"\n Métricas do modelo {nome_modelo}:", metricas)

In [ ]:
caminho_collection = "/content/drive/MyDrive/dataset/nlp/semantica/msmarco/portuguese_collection.tsv"
caminho_queries = "/content/drive/MyDrive/dataset/nlp/semantica/msmarco/portuguese_queries.dev.tsv"
caminho_qrels = "/content/drive/MyDrive/dataset/nlp/semantica/msmarco/qrels.dev.tsv"
caminho_bm25 = "/content/drive/MyDrive/dataset/nlp/semantica/msmarco/run.bm25_portuguese-msmarco.txt"

df_docs = pd.read_csv(caminho_collection, sep='\t', quoting=csv.QUOTE_NONE,header=None, names=['doc_id', 'document'])
df_queries = pd.read_csv(caminho_queries, sep='\t', header=None, names=['query_id', 'query'])
df_qrels = pd.read_csv(caminho_qrels, sep='\t', header=None, names=['query_id', '0', 'doc_id', 'relevance'])
df_bm25 = pd.read_csv(caminho_bm25, sep='\t', header=None, names=['query_id', 'doc_id', 'rank'])


In [ ]:
contagem = df_qrels['query_id'].value_counts()
qtd_2_ocorrencias = len(contagem[contagem == 2])
qtd_3_ocorrencias = len(contagem[contagem == 3])
qtd_4_ocorrencias = len(contagem[contagem == 4])
qtd_5_ocorrencias = len(contagem[contagem == 5])
print(f"Número de identificadores com mais de uma ocorrência")
print(f"2 ocorrências: {qtd_2_ocorrencias}")
print(f"3 ocorrências: {qtd_3_ocorrencias}")
print(f"4 ocorrências: {qtd_4_ocorrencias}")
print(f"5 ocorrências: {qtd_5_ocorrencias}")

Número de identificadores com mais de uma ocorrência
2 ocorrências: 2690
3 ocorrências: 355
4 ocorrências: 72
5 ocorrências: 16


In [ ]:
#len(df_docs)
df_docs.head(5)

,doc_id,document
0,0,A presença de comunicação entre mentes científ...
1,1,O Projeto Manhattan e sua bomba atômica ajudar...
2,2,Ensaio sobre o Projeto Manhattan - O Projeto M...
3,3,O Projeto Manhattan era o nome de um projeto r...
4,4,"versões de cada volume, bem como sites complem..."


In [ ]:
df_qrels_2_ocorrencias = contagem[contagem == 2]
df_qrels_2_ocorrencias.head(3)
df_qrels_2_ocorrencias_exemplos = df_qrels[df_qrels['query_id'].isin(df_qrels_2_ocorrencias.index)]
df_qrels_2_ocorrencias_exemplos.head(4)

df_qrels_3_ocorrencias = contagem[contagem == 3]
df_qrels_3_ocorrencias.head(3)
df_qrels_3_ocorrencias_exemplos = df_qrels[df_qrels['query_id'].isin(df_qrels_3_ocorrencias.index)]
df_qrels_3_ocorrencias_exemplos.head(6)

,query_id,0,doc_id,relevance
13,114414,0,7067093,1
14,114414,0,7067094,1
15,114414,0,7067099,1
214,30178,0,7071029,1
215,30178,0,5148105,1
216,30178,0,6481004,1


In [ ]:
df_queries_train = pd.read_csv("/content/drive/MyDrive/dataset/nlp/semantica/msmarco/portuguese_queries.train.tsv", sep='\t', header=None, names=['query_id', 'query'])
df_queries_small = pd.read_csv("/content/drive/MyDrive/dataset/nlp/semantica/msmarco/portuguese_queries.dev.small.tsv", sep='\t', header=None, names=['query_id', 'query'])


In [ ]:
comum = pd.merge(df_queries_train, df_queries_small, on='query_id')['query_id'].unique()
print(len(comum))

0


In [ ]:
qrels_query_ids = df_qrels['query_id'].unique()
queries_not_in_qrels = df_queries[~df_queries['query_id'].isin(qrels_query_ids)]
print("Registros de df_queries que não estão em df_qrels:")
print(queries_not_in_qrels)

queries_train_not_in_qrels = df_queries_train[~df_queries_train['query_id'].isin(qrels_query_ids)]
print("\nRegistros de df_queries_train que não estão em df_qrels:")
print(queries_train_not_in_qrels)

Registros de df_queries que não estão em df_qrels:
        query_id                                              query
2        1048580                         o que é desperdício de pcb
3        1048581                                      o que é pbis?
6        1048584  Qual é a faixa de pagamento para especialista ...
8        1048586                     o que é doença da gengiva paul
12        524302                        base de custo de tesouraria
...          ...                                                ...
101082   1048546                               quem joga sam no lmn
101083   1048548           o que é um exemplo de relação de fixação
101088    480594                     preço do cobre por onça, libra
101089    524271          efeitos colaterais da trazodona para cães
101092    524285               significado de inclinação da esteira

[45515 rows x 2 columns]

Registros de df_queries_train que não estão em df_qrels:
        query_id                                 